## はじめに
これは Kaggle の [House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) 練習用ノートブックです。このノートブックでは、AI Platform Notebooksでモデルを学習し、Kaggleに提出する方法を学習します。

はじめに、Cloud AI NotebooksインスタンスにKaggleコマンドをインストールし、データセットを取得します。

In [1]:
pip install kaggle

    100% |████████████████████████████████| 61kB 3.0MB/s ta 0:00:011
    100% |████████████████████████████████| 163kB 4.8MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 4.0MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 8.3MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 9.5MB/s eta 0:00:01
    100% |████████████████████████████████| 122kB 5.3MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 8.2MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 8.5MB/s eta 0:00:01
    100% |████████████████████████████████| 143kB 6.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for kaggle ... done
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
  Running setup.py bdist_wheel for python-slugify ... done
  Stored in directory: /root/.cache/pip/wheels/11/94/81/312969455540cb0e6a773e5d68a73c14128bfdfd4a7969bb4f
Successfully built kaggle python-slugify
  Found ex

In [2]:
# Move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

ここでは、最小限のデータ読み込みから提出の流れをわかりやすくするため、モデル自体はシンプルなRandomForestで行っています。 

In [4]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

  0%|                                                | 0.00/199k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 199k/199k [00:00<00:00, 84.9MB/s]


In [5]:
!unzip house*

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Read the data
train = pd.read_csv('./train.csv')

# pull data into target (y) and predictors (X)
train_y = train.SalePrice
predictor_cols = ['LotArea', 'OverallQual', 'YearBuilt', 'TotRmsAbvGrd']

# Create training predictors data
train_X = train[predictor_cols]

my_model = RandomForestRegressor()
my_model.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

トレーニングデータに加えて、テストデータがあります。 これは、一般的にはtest.csvというタイトルのファイル名で保存されます。 このデータには、予測対象となるターゲット（y）列は含まれません。

In [7]:
# Read the test data
test = pd.read_csv('./test.csv')
# Treat the test data in the same way as training data. In this case, pull same columns.
test_X = test[predictor_cols]
# Use the model to make predictions
predicted_prices = my_model.predict(test_X)
# We will look at the predicted prices to ensure we have something sensible.
print(predicted_prices)

[131570.   184415.   156899.4  ... 148361.36 163127.27 252296.36]


### Submission ファイルを準備する
Submission は CSV ファイルとして提出します。 Submissionファイルには、通常IDと予測対象、2つの列があります。 IDはテストデータから取得されます（住宅データの場合は文字列 'Id'です）。 予測列には、ターゲットフィールドの名前が使用されます。

このデータを使用してDataFrameを作成し、データフレームのto_csvメソッドを使用して送信ファイルを書き込みます。 Pandasがcsvファイルに別の列を追加しないように、引数index = Falseを明示的に含めます。

In [8]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

KaggleコマンドでSubmitします。

In [9]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Submit from Cloud AI Notebooks"

100%|██████████████████████████████████████| 21.5k/21.5k [00:01<00:00, 13.2kB/s]
Successfully submitted to House Prices: Advanced Regression Techniques